<h1>MongoDB</h1><h4>Python API and Jupyter + nbconvert demo</h4>

<i>MongoDB</i> is an example of a <i>Document-Oriented Database</i>. That means that instead of working with tables of normalized data like in a relational database, you work with complete chunks of information called <i>documents</i> which are stored inside <i>collections</i> of similar data.

For example, a document in a clinical setting might describe a patient.

In [ ]:
{
    "Name": "Jon Smith",
    "Age": 32,
    "Children": ["Fred", "Jane"],
    "LastVisit": {
        "Time": "2018-02-05 14:49",
        "Symptoms": ["Headache"]
    }
}

This document would likely belong to a <b>collection</b> of patients.

The syntax here is both a valid Python dictionary and an example of a <a href="https://www.json.org/">JSON</a> document.

To use MongoDB from Python, we must first connect to the database server:

In [ ]:
from pymongo import MongoClient
mongodb = MongoClient(username='USERNAME',
                      password='PASSWORD',
                      authSource='DBNAME')
mydb = mongodb['DBNAME']

... where USERNAME, PASSWORD, and DBNAME are replaced with the appropriate values.

Once connected, you can change your password via:

In [ ]:
mydb.command('updateUser', netid, pwd='12345')

If you are running locally with no security (don't do this), you can skip the authentication:

In [3]:
from pymongo import MongoClient
mongodb = MongoClient()

Let's create a database called <b>medical_history</b> for storing medical history information.

Since we are experimenting, we will begin by dropping (deleting) any previous database of that name:

In [13]:
mongodb.drop_database('medical_history')

And now we can create it simply by referencing it using either the [] or dot notation:

In [14]:
mydb = mongodb.medical_history  # same as mongodb['medical_history']

In [15]:
type(mydb)

pymongo.database.Database

Now that we can connect to our database, there are four key types of operations, known by the acronym <a href="https://en.wikipedia.org/wiki/Create,_read,_update_and_delete">CRUD</a>:
<ul>
<li><i>C</i>reate</li>
<li><i>R</i>ead</li>
<li><i>U</i>pdate</li>
<li><i>D</i>elete</li>
</ul>

<h1>Create</h1>

To enter a document in our database, we must first create a collection. Since MongoDB is by default schemaless, this can be done simply by referring to the collection by name:

In [16]:
patient_info = mydb.patient_info
type(patient_info)

pymongo.collection.Collection

Schemas for a collection can be specified using the draft <a href="http://json-schema.org/">JSON Schema</a> standard, e.g.

In [20]:
mydb.create_collection('patients', validator={
    '$jsonSchema': {
        'required': ['age', 'sex'],
        'properties': {
            'age': {
                'bsonType': 'int',
                'minimum': 0,
                'maximum': 100
            },
            'sex': {
                'bsonType': 'string',
                'enum': ['male', 'female']
            }
        }
    }
})
patients = mydb.patients

We can insert a single document into a collection using <b>insert_one</b>:

In [21]:
response = patient_info.insert_one(
    {
        'Last': 'Shingler',
        'First': 'Cynthia',
        'age': 42,
        'sex': 'female',
        'conditions': ['High blood pressure']
    }
)

The insert_one call returns an object that can be queried to obtain the automatically generated <b>_id</b> field of the document. (This is indexed by default.) Alternatively, an <b>_id</b> could have been specified in the document being inserted; if so, it would have been required to be unique.

In [27]:
response.inserted_id

ObjectId('5a78c353a2db802558fa8b02')

Attempting to insert a document that does not match the schema provided (if any) raises an exception. Recall that we previously required documents in <b>patients</b> to have both an <b>age</b> and a <b>sex</b>:

In [28]:
patients.insert_one({'Last': 'Cain', 'First': 'Winona'})

WriteError: Document failed validation

Which of the following commands will succeed? If you are viewing this interactively, run each cell to test your predictions.

In [ ]:
patient_info.insert_one({'last': 'Eder', 'first': 'Debra',
                         'age': 50, 'sex': 'female'})

In [ ]:
patients.insert_one({'last': 'Cassidy', 'first': 'Bernie',
                     'age': 1978, 'sex': 'male'})

In [ ]:
patient_info.insert_one({'Last': 'Murphy', 'First': 'Manuel',
                         'Age': 25})

In [ ]:
patients.insert_one({'last': 'Murphy', 'first': 'Manuel',
                     'age': 25, 'sex': 'california'})

We can also <b>insert_many</b> documents at a time from a Python iterable (e.g. a list):

In [30]:
patients.insert_many([
    {'last': 'Bowman', 'first': 'Deana', 'age': 32, 'sex': 'female'},
    {'last': 'Walker', 'first': 'Marcos', 'age': 79, 'sex': 'male'},
    {'last': 'Dimiceli', 'first': 'Scott', 'age': 67, 'sex': 'male'}
])

By default, the items are inserted in order; this means that if a document fails validation, no documents later in the list will be inserted.

Including the optional keyword argument <b>ordered=False</b> causes every document insertion to be attempted.

To have an interesting collection of data to work with, lets drop the collections that we have right now, and recreate patients with synthetic data from <a href="patients.csv">patients.csv</a>:

In [31]:
mydb.drop_collection('patients')
mydb.drop_collection('patient_info')

{u'nIndexesWas': 1, u'ns': u'medical_history.patient_info', u'ok': 1.0}

Reintroducing the schema rules...

In [32]:
mydb.create_collection('patients', validator={
    '$jsonSchema': {
        'required': ['age', 'sex'],
        'properties': {
            'age': {
                'bsonType': 'int',
                'minimum': 0,
                'maximum': 100
            },
            'sex': {
                'bsonType': 'string',
                'enum': ['male', 'female']
            }
        }
    }
})
patients = mydb.patients

Now let's load the data, dropping any entries that do not validate (data-entry errors are common):

In [34]:
import pymongo
with open("people.csv") as f:
    for line in f:
        patient_id, last, first, age, sex = line.strip().split(',')
        patient_id = int(patient_id)
        age = int(age)
        try:
            mydb.patients.insert_one({
                '_id': patient_id,
                'last': last,
                'first': first,
                'age': age,
                'sex': sex
            })
        except pymongo.errors.WriteError:
            # invalid entry
            pass

We'll also load in artificial conditions data from <a href="diagnoses.csv">diagnoses.csv</a>. Note that this is not patient-centric; we'll fix that later.

In [36]:
with open('diagnoses.csv') as f:
    for line in f:
        patient, condition = line.strip().split(',')
        patient = int(patient)
        mydb.conditions.insert_one({'patient': patient, 'condition': condition})

The data may be viewed, queried, and edited using the graphical <b>MongoDB Compass</b> tool. This also provides a quick way to make sure the data was imported correctly.

<img src="mongodb-compass.png"/>

<h1>Read</h1>

We will discuss three forms of reading from the database: <ul><li>counting documents</li><li>finding documents</li><li>aggregating documents</li></ul>

<h1>Counting documents</h1>

The most basic question we can ask is how many documents are in a collection:

In [37]:
patients.count()

9766

We can specify a document to match against to count only a subset; e.g. to find how many patients have the last name "Smith":

In [38]:
patients.count({'last': 'Smith'})

123

Now it's your turn. How many patients have the first name "Robert"? 

How many patients are female?

To find a single document matching a query, use <b>find_one</b>:

In [41]:
patients.find_one({'_id': 8509})

{u'_id': 8509,
 u'age': 89,
 u'first': u'Raymond',
 u'last': u'Phillips',
 u'sex': u'male'}

To find all documents matching a query, use <b>find</b>. This returns an iterable that returns the data from the database as needed; e.g. to find all of patient 8509's conditions:

In [42]:
for document in mydb.conditions.find({'patient': 8509}):
    print(document['condition'])

Diabetes
Periodontal disease
Drug abuse
High blood pressure
Androgenetic alopecia
Alzheimer's


Your turn.

Find the first names of all patients aged 89; my answer is shown below.

Lisa, Mable, Melody, Deborah, Mark, Miriam, Ivan, Leslie, Crystal, Jorge, 
David, Vickie, Michelle, Norman, Anthony, Evita, Paul, Roy, Anna, Matthew, 
Eva, Mary, Charlotte, Angela, Robert, Marion, Sofia, Clayton, Juanita, Henry, 
Michael, Martha, Marie, Aaron, Sandra, Sarah, Kim, Antonio, Maxine, Sara, 
Matthew, Dixie, Delores, Celeste, Olga, Raymond, Robert, Arthur, Linda, Kayla, 
Donna, Kenneth, William, Stephanie, Charles, Luke


We can search against multiple fields simultaneously:

In [74]:
list(patients.find({'last': 'Smith', 'first': 'John'}))

[{u'_id': 1420,
  u'age': 9,
  u'first': u'John',
  u'last': u'Smith',
  u'sex': u'male'},
 {u'_id': 6364,
  u'age': 55,
  u'first': u'John',
  u'last': u'Smith',
  u'sex': u'male'},
 {u'_id': 7430,
  u'age': 69,
  u'first': u'John',
  u'last': u'Smith',
  u'sex': u'male'}]

MongoDB supports a number of <a href="https://docs.mongodb.com/manual/reference/operator/query/">query operators</a>, all prefixed with a $ sign. These can be used with both <b>count</b> and <b>find</b>.

For example, to find out how many patients are over age 80:

In [76]:
patients.count({'age': {'$gt': 80}})

542

Patients named Smith who are either over age 80 or under 10:

In [80]:
patients.count({'last': 'Smith',
                '$or': [
                    {'age': {'$gt': 80}},
                    {'age': {'$lt': 10}}
                ]})

17

We can find the distinct set of values for a given field in a search result by combining <b>find</b> with <b>distinct</b>:

In [81]:
patients.find({'last': 'Smith', 'first': 'John'}).distinct('age')

[9, 55, 69]

Your turn: 

What are the distinct conditions (field name: <b>condition</b>) in the <b>mydb.conditions</b> collection? (Hint: there are 10.)

<h1>Aggregation</h1>

Aggregation is a general framework that allows you to query documents, cross-reference documents (think: left outer joins in SQL), group data, do calculations, rename fields, create new collections, etc. It is related to the concept of a map-reduce. For more, see the <a href="http://api.mongodb.com/python/current/examples/aggregation.html">PyMongo Aggregation</a> documentation.

We can create a new virtual collection that groups the patients by last name and counts the number of matches:

In [88]:
for i, data in enumerate(mydb.patients.aggregate([
    {'$group': {'_id': '$last', 'count': {'$sum': 1}}},
])):
    print('{_id:20}{count:10}'.format(**data))
    if i == 4: break

Canterbury                   1
Roundtree                    1
Guest                        1
Nasser                       1
Gillis                       1


If we are looking for a specific name, we can use the <b>$match</b> operator to narrow the results:

In [90]:
for i, data in enumerate(mydb.patients.aggregate([
    {'$match': {'last': 'Jones'}},
    {'$group': {'_id': '$last', 'count': {'$sum': 1}}},
])):
    print('{_id:20}{count:10}'.format(**data))

Jones                       71


We do the <b>\$match</b> before the <b>\$group</b> to minimize the amount of data that must be grouped.

If instead we want to know the most common names, we can do a <b>$sort</b> on the count:

In [91]:
for i, data in enumerate(mydb.patients.aggregate([
    {'$group': {'_id': '$last', 'count': {'$sum': 1}}},
    {'$sort': {'count': -1}}
])):
    print('{_id:20}{count:10}'.format(**data))
    if i == 4: break

Smith                      123
Johnson                    101
Williams                    90
Jones                       71
Davis                       61


A <b>$lookup</b> lets us insert matches from one collection (below: conditions) into another (below: patients) based on a matching value (below: patient _id):

In [94]:
from pprint import pprint
for document in mydb.patients.aggregate([
    {
         '$lookup': {
             'from': 'conditions',
             'localField': '_id',
             'foreignField': 'patient',
             'as': 'conditions'
         },
    }]):
    pprint(document)
    break


{u'_id': 1,
 u'age': 51,
 u'conditions': [{u'_id': ObjectId('5a78cc14a2db802558faad51'),
                  u'condition': u'Periodontal disease',
                  u'patient': 1},
                 {u'_id': ObjectId('5a78cc14a2db802558fabdf0'),
                  u'condition': u'Androgenetic alopecia',
                  u'patient': 1}],
 u'first': u'Jimmy',
 u'last': u'Wise',
 u'sex': u'male'}


In practice, we rarely want the full document. Instead, we <b>$project</b> the combined entity to get just the fields we want:

In [95]:
for document in mydb.patients.aggregate([
    {
         '$lookup': {
             'from': 'conditions',
             'localField': '_id',
             'foreignField': 'patient',
             'as': 'conditions'
         },
    },
    {
        '$project': {
            'conditions': '$conditions.condition',
            'first': True,
            'last': True,
            'age': True,
            'sex': True,
            '_id': True
        }
    }
]):
    pprint(document)
    break

{u'_id': 1,
 u'age': 51,
 u'conditions': [u'Periodontal disease', u'Androgenetic alopecia'],
 u'first': u'Jimmy',
 u'last': u'Wise',
 u'sex': u'male'}


To save the results of an aggregation into a new collection, use <b>$out</b> as the last step:

In [96]:
mydb.patients.aggregate([
    {
         '$lookup': {
             'from': 'conditions',
             'localField': '_id',
             'foreignField': 'patient',
             'as': 'conditions'
         },
    },
    {
        '$project': {
            'conditions': '$conditions.condition',
            'first': True,
            'last': True,
            'age': True,
            'sex': True,
            '_id': True
        }
    },
    {
        '$out': 'patient_data'
    }
])

To speed up searches (at the cost of slower inserts), it is advantageous to <b>create_index</b> on a collection's field. e.g. to allow quickly seaching on conditions:

In [97]:
mydb.patient_data.create_index('conditions')

u'conditions_1'

Your turn.

Now that we have this combined, patient-centric collection, we can see how many patients have been diagnosed with Diabetes and Alzheimer's in that order and no other conditions:

In [101]:
mydb.patient_data.count({'conditions': ['Diabetes', "Alzheimer's"]})

4

Using the <b>\$and</b>, <b>\$match</b>, and <b>\$sort</b> operators with aggregation, find the first four names of people with both conditions (and potentially others) sorted alphabetically. (Hint: there are 98 such people total.)

We can group on conditions to count the prevalence of condition patterns:

In [103]:
for i, data in enumerate(mydb.patient_data.aggregate([
    {'$group': {'_id': '$conditions', 'count': {'$sum': 1}}},
    {'$sort': {'count': -1}}
])):
    print('{_id:60}{count:10}'.format(**data))
    if i == 5: break

[]                                                                2467
[u'Periodontal disease']                                          1485
[u'High blood pressure']                                           683
[u'Androgenetic alopecia']                                         350
[u'Migraines']                                                     287
[u'High blood pressure', u'Periodontal disease']                   279


The 279 people diagnosed with High blood pressure and periodontal disease (in that order) are not counted in the 683 with high blood pressure.

To count the conditions individually, we can first <b>$unwind</b> the list of conditions -- splitting each document into multiple documents, one per condition -- and then counting:

In [104]:
for i, data in enumerate(mydb.patient_data.aggregate([
    {'$unwind': '$conditions'},
    {'$group': {'_id': '$conditions', 'count': {'$sum': 1}}},
    {'$sort': {'count': -1}}
])):
    print('{_id:60}{count:10}'.format(**data))
    if i == 5: break

Periodontal disease                                               4244
High blood pressure                                               3114
Androgenetic alopecia                                             2209
Migraines                                                         1031
Diabetes                                                           873
Alcohol abuse                                                      550


Your turn

What are the top six conditions of women aged 20 in this dataset?

Periodontal disease                             27
High blood pressure                              8
Migraines                                        6
Alcohol abuse                                    4
Obsessive-compulsive disorder                    3
Diabetes                                         2


<h1>Updating</h1>

The <b>update_many</b> method on a collection takes a search query, and an update rule. Various update rules are available, including <b>\$set</b>, <b>\$inc</b>, and <b>\$mul</b>.

For example, to rename the condition <b>Alzheimer's</b> to <b>Alzheimer's disease</b>:

In [108]:
mydb.patient_data.update_many(
    {'conditions': "Alzheimer's"},
    {'$set': {'conditions.$': "Alzheimer's disease"}}
)

Note that patient 8509 is currently 89 years old:

In [109]:
mydb.patient_data.find_one({'_id': 8509})['age']

89

Suppose it is his birthday. Let's increase his age by 1:

In [112]:
mydb.patient_data.update_many(
    {'_id': 8509},
    {'$inc': {'age': 1}}
)

In [113]:
mydb.patient_data.find_one({'_id': 8509})['age']

90

<h1>Delete</h1>

To remove a document from a collection, we use <b>delete_one</b> or <b>delete_many</b>.

In [114]:
mydb.patient_data.count({'last': 'Jones'})

71

In [115]:
mydb.patient_data.delete_one({'last': 'Jones'})
mydb.patient_data.count({'last': 'Jones'})

70

In [116]:
mydb.patient_data.delete_many({'last': 'Jones'})
mydb.patient_data.count({'last': 'Jones'})

0

We can delete a collection by dropping it:

In [117]:
mydb.drop_collection('patient_data')

{u'nIndexesWas': 2, u'ns': u'medical_history.patient_data', u'ok': 1.0}

and delete a database by dropping it:

In [119]:
mongodb.drop_database('medical_history')